In [7]:
import pandas as pd
import datetime
import json

In [81]:
END_YEAR = datetime.datetime.now().year - 1
CHALLENGER_PRIZE_MONEY_FACTOR = 5
MIN_EARNINGS = 100000

In [82]:
def clean_tournaments_string(tournaments_string):
    return tournaments_string.replace("'", '"')

In [83]:
experimental_data = []
total_additional_prize_money = 0

df = pd.read_csv(f"../data/players/combined/{END_YEAR}.gz")
for i, _ in df.iterrows():
    player_prize_money = df.loc[i, "total_prize_money"]

    clean_singles = clean_tournaments_string(df.loc[i, "singles_tournaments"])
    singles_tournaments = json.loads(clean_singles[clean_singles.find("["):])

    for singles_tournament in singles_tournaments:
        if "challenger" in singles_tournament["tournament_type"].lower():
            tournament_prize_money = int(singles_tournament["prize_money"][1:])
            additional_prize_money = tournament_prize_money * (CHALLENGER_PRIZE_MONEY_FACTOR - 1)

            player_prize_money += additional_prize_money
            total_additional_prize_money += additional_prize_money

    clean_doubles = clean_tournaments_string(df.loc[i, "doubles_tournaments"])
    doubles_tournaments = json.loads(clean_doubles[clean_doubles.find("["):])
    for doubles_tournament in doubles_tournaments:
        if "challenger" in doubles_tournament["tournament_type"].lower():
            tournament_prize_money = int(doubles_tournament["prize_money"][1:])
            additional_prize_money = tournament_prize_money * (CHALLENGER_PRIZE_MONEY_FACTOR - 1)

            player_prize_money += additional_prize_money
            total_additional_prize_money += additional_prize_money

    experimental_data.append({
        "player_id": df.loc[i, "player_id"],
        "type": df.loc[i, "type"],
        "total_prize_money": player_prize_money,
        "singles_tournaments": singles_tournaments,
        "doubles_tournaments": doubles_tournaments
    })

experimental_df = pd.DataFrame(experimental_data)
print("Total additional prize money:", total_additional_prize_money)


Total additional prize money: 41457480


In [84]:
data = []
total = 0
singles = 0
doubles = 0

for i, row in experimental_df.iterrows():
    if row["total_prize_money"] >= MIN_EARNINGS:
        total += 1
        if row["type"] == "singles":
            singles += 1
        elif row["type"] == "doubles":
            doubles += 1

data.append({
    "year": END_YEAR,
    "total": total,
    "singles": singles,
    "doubles": doubles,
})

results = pd.DataFrame(data)

In [85]:
results

,year,total,singles,doubles
0,2022,422,290,132
